# Import and Set-Up

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import numpy as np
import pandas as pd
import gc
import csv

import warnings
warnings.filterwarnings('ignore')

import os
import datetime
import time
import glob
import os.path as osp
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline


import torch
import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils import data as D
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
print("total GPU: %d" %(torch.cuda.device_count()))

cpu
total GPU: 0


In [28]:
import random

In [29]:
!mkdir /content/dataset
!unzip /content/drive/MyDrive/New\ Exps/Dataset/CompCars.zip -d /content/dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/part/77/159/2010/2/510208d2e16924.jpg  
  inflating: /content/dataset/part/77/159/2010/3/56b85f75643fcd.jpg  
  inflating: /content/dataset/part/77/159/2010/3/6ce67ffc8ee4f5.jpg  
  inflating: /content/dataset/part/77/159/2010/4/361ab40804d981.jpg  
  inflating: /content/dataset/part/77/159/2010/4/de3c9b5bbb5827.jpg  
  inflating: /content/dataset/part/77/159/2010/5/c2c25e35071415.jpg  
  inflating: /content/dataset/part/77/159/2010/5/da03fe61f467dd.jpg  
  inflating: /content/dataset/part/77/159/2010/6/8fca577c2ba86d.jpg  
  inflating: /content/dataset/part/77/159/2010/6/c2e58551864677.jpg  
  inflating: /content/dataset/part/77/159/2010/7/5e9e144e050cf0.jpg  
  inflating: /content/dataset/part/77/159/2010/7/e1696de65b0894.jpg  
  inflating: /content/dataset/part/77/159/2010/8/062e405280e0c7.jpg  
  inflating: /content/dataset/part/77/159/2011/1/0ff534924111a2.jpg  
  inflating: /content/dataset/part/77/1

In [33]:
import os
from PIL import Image
import scipy.io
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class CompCarsDataset(Dataset):
    def __init__(self, root_dir='/content/dataset/', transform=None):
        """
        Args:
            root_dir (str): Path to dataset (train or test folder).
            mat_file (str): Path to .mat file containing class names.
            transform (callable, optional): Transformations to apply.
        """
        self.root_dir = root_dir
        self.image_dir = root_dir + 'image/'
        self.label_dir = root_dir + 'label/'
        self.split_dir = root_dir + 'train_test_split/'
        self.misc_dir = root_dir + 'misc/'

        print(self.train_images[:10])
        assert False
        self.transform = transform
        self.makes, self.models = self._load_class_names(root_dir + '/misc/make_model_name.mat')
        print(len(self.makes))
        print(len(self.models))

        self.image_paths = []
        self.labels = []

        self.alphabetical_to_original_label = {}

        # Collect all image paths and their corresponding labels
        self.no_classlabels = -1
        self.no_makes = -1
        self.no_models = -1
        for make in os.listdir(self.image_dir):
            make_path = os.path.join(self.image_dir, make)
            self.no_makes += 1
            for model in os.listdir(make_path):
                model_path = os.path.join(make_path, model)
                self.no_models += 1
                for year in os.listdir(model_path):
                    year_path = os.path.join(model_path, year)
                    self.no_classlabels += 1
                    self.alphabetical_to_original_label[self.no_classlabels] = {'index': self.no_classlabels, 'class': self.models[self.no_models], \
                                                                                'super_index': self.no_makes, 'superclass': self.makes[self.no_makes]}
                    for img_name in os.listdir(year_path):
                      img_path = os.path.join(year_path, img_name)
                      self.image_paths.append(img_path)
                      self.labels.append(self.no_classlabels)

        combined = list(zip(self.image_paths, self.labels))
        random.shuffle(combined)
        self.image_paths, self.labels = zip(*combined)
        print(self.no_classlabels)

    def _load_class_names(self, mat_file):
        data = scipy.io.loadmat(mat_file)
        makes_names = data.get('make_names')
        if makes_names is not None:
            makes =  [name[0] for name in makes_names.flatten()]
        else:
            raise ValueError("Could not find 'make_names' key in .mat file")
        model_names = data.get('model_names')
        # print(model_names.flatten())
        if model_names is not None:
            models =  [name for name in model_names.flatten()]
        else:
            raise ValueError("Could not find 'model_names' key in .mat file")
        # print(models)
        return makes, models

    def get_alphabetical_to_original_label(self):
        return self.alphabetical_to_original_label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert("RGB")  # Open image

        if self.transform:
            image = self.transform(image)  # Apply transformations

        return image, label

dataset = CompCarsDataset()


['78/1/2010/439374a1456969.jpg', '78/1/2010/6c5730eb35dbfc.jpg', '78/1/2014/a2b9bfe21409f4.jpg', '78/1/2013/043602fa21a621.jpg', '78/1/2012/ccc640bfa095e3.jpg', '78/1/2014/bd93fbe63366cd.jpg', '78/1/2014/6f527c039c1ac3.jpg', '78/1/2012/4f0e2c4390e70a.jpg', '78/1/2010/aa93b7c8310b58.jpg', '78/1/2013/64ef1f35270db5.jpg']


AssertionError: 

In [31]:
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Paths
root_dir = "compcars/train"  # Adjust as needed
mat_file = "compcars/misc/class_list.mat"  # Adjust as needed

# Create dataset
train_dataset = CompCarsDataset(root_dir=root_dir, mat_file=mat_file, transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Check a sample
for imgs, labels in train_loader:
    print(imgs.shape, labels.shape)
    break

TypeError: CompCarsDataset.__init__() got an unexpected keyword argument 'mat_file'